# Autism Spectrum Disorder (ASD) Stress Prediction

### Sensor Data-Based Classification Using Machine Learning

Here we train a **Random Forest model** to classify ASD-related stress levels using sensor data.

## Import necessary libraries


In [19]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
import joblib, pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Load the dataset

In [2]:
data = pd.read_csv('ASD_data.csv')


In [3]:
data.head()

,gsr_max,gsr_min,gsr_mean,gsr_sd,hrate_mean,temp_avg,fear,angry,happy,sad,class
0,2.32,1.63,1.730828,0.101924,84.9300,38.8,3,2,1,2,3
1,1.93,1.63,1.723939,0.113226,79.5426,38.7,1,1,1,5,3
2,2.81,1.93,2.189154,0.289796,98.9055,37.9,1,1,4,1,2
3,3.00,2.32,2.731613,0.270726,100.8939,38.9,2,2,1,4,3
4,7.20,3.10,4.141620,1.020889,105.0950,39.0,2,2,1,5,3


In [4]:
data.shape

(170, 11)


## features and target variable

In [6]:
features = ['gsr_max', 'gsr_min', 'gsr_mean', 'gsr_sd', 'hrate_mean', 'temp_avg']
target = 'class'
X = data[features]
y = data[target]

## Check missing values

In [8]:
missing_values = X.isnull().sum()
print("Missing values before handling:",missing_values)

Missing values before handling: gsr_max       0
gsr_min       0
gsr_mean      0
gsr_sd        1
hrate_mean    0
temp_avg      0
dtype: int64


## Handle missing values by filling with column mean

In [10]:

X = X.fillna(X.mean())
print("Missing values after handling:", X.isnull().sum().sum())

Missing values after handling: 0


## Outlier Detection and Removal using IQR

In [ ]:
Q1 = X.quantile(0.25)
Q3 = X.quantile(0.75)
IQR = Q3 - Q1

outliers = ((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).sum()
print("Outliers detected per feature:", outliers)


Outliers detected per feature: gsr_max       12
gsr_min       12
gsr_mean       8
gsr_sd        16
hrate_mean     4
temp_avg       0
dtype: int64


### Remove rows containing outliers

In [ ]:

X_filtered = X[~((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).any(axis=1)]
y_filtered = y[X_filtered.index]

print("Data shape after outlier removal:", X_filtered.shape)

Data shape after outlier removal: (141, 6)


## Normalize the feature data

In [13]:
scaler = preprocessing.MinMaxScaler()
X_scaled = scaler.fit_transform(X_filtered)
print("Data normalized using MinMaxScaler.")

Data normalized using MinMaxScaler.


## Split data into training and testing sets

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_filtered, test_size=0.2, random_state=42)
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)

Training data shape: (112, 6)
Testing data shape: (29, 6)


## Train the RandomForest model

In [15]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
print("Model training complete.")

Model training complete.


## Make predictions and evaluate the model

In [16]:
y_pred = model.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred, average='weighted')
recall = metrics.recall_score(y_test, y_pred, average='weighted')
f1 = metrics.f1_score(y_test, y_pred, average='weighted')
print("Model Evaluation Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Model Evaluation Metrics:
Accuracy: 0.97
Recall: 0.97
F1 Score: 0.97


## Save the trained model and scaler

In [20]:
# joblib.dump(model, 'auticare_model.pkl')
with open("auticare_model.pkl", "wb") as f:
    pickle.dump(model, f)
joblib.dump(scaler, 'scaler.pkl')
print("Model and scaler saved successfully.")




Model and scaler saved successfully.
